In [ ]:
!python3 -m pip install --upgrade pip
!python3 -m pip install tensorboard==1.14

In [ ]:
!apt-get update && apt-get install -y curl

In [ ]:
!curl https://install.zerotier.com | bash
!zerotier-cli status

In [ ]:
!zerotier join af78bf943689a51e

In [1]:
import torch
import base64
import io
import cv2
import numpy as np
import PIL.Image



def img_data_to_pil(img_data):
    f = io.BytesIO()
    f.write(img_data)
    img_pil = PIL.Image.open(f)
    return img_pil


def img_data_to_arr(img_data):
    img_pil = img_data_to_pil(img_data)
    img_arr = np.array(img_pil)
    return img_arr


def img_b64_to_arr(img_b64):
    img_data = base64.b64decode(img_b64)
    img_arr = img_data_to_arr(img_data)
    return img_arr


def img_pil_to_data(img_pil):
    f = io.BytesIO()
    img_pil.save(f, format="PNG")
    img_data = f.getvalue()
    return img_data


def img_arr_to_b64(img_arr):
    img_pil = PIL.Image.fromarray(img_arr)
    f = io.BytesIO()
    img_pil.save(f, format="PNG")
    img_bin = f.getvalue()
    if hasattr(base64, "encodebytes"):
        img_b64 = base64.encodebytes(img_bin)
    else:
        img_b64 = base64.encodestring(img_bin)
    return img_b64

def labelme_to_yolo(proj_name, labels):
    # make folder structure
    src_dir = osp.join(proj_name, 'src')
    images_dir = osp.join(proj_name, f'images/{proj_name}')
    labels_dir = osp.join(proj_name, f'labels/{proj_name}')
    data_dir = proj_name

    for fold in ([src_dir, 
                      images_dir, 
                      labels_dir, 
                      data_dir]):
        if not osp.isdir(fold):
            os.makedirs(fold)
                

    for file_name in os.listdir(src_dir):
        if file_name.endswith('.json'):
            name = file_name.split('.')[0]
            path = osp.join(src_dir, file_name)
            with open(path,'r') as f:
                ann = json.load(f)

            # save image
            img  = img_b64_to_arr(ann['imageData']).astype(np.uint8)
            img_name = name + '.bmp'
            dst_img_path = osp.join(images_dir, img_name)
            cv2.imwrite(dst_img_path, img)
        
            # save labels
            img_w = ann['imageWidth']
            img_h = ann['imageHeight']
        
            yolo_ann = ''
            for shape in ann['shapes']:
                label = shape['label']
                if label not in labels:
                    continue
                print(label)  
                l_id = labels.index(label)
                pts = shape['points']
                pts = np.asarray(pts)
                xmin, ymin = np.min(pts, axis=0).tolist()
                xmax, ymax = np.max(pts, axis=0).tolist()
                w, h = xmax - xmin, ymax - ymin
                yolo_ann += f'{l_id} {(xmin+xmax)/2/img_w:.6f} {(ymin+ymax)/2/img_h:.6f} {w/img_w:.6f} {h/img_h:.6f}\n'
        
            label_name = name + '.txt'
            label_path = osp.join(labels_dir, label_name)
            with open(label_path, 'w') as f:
                print(yolo_ann)
                f.write(yolo_ann)
                
    # create data file like
    #classes=3
    #train=../trunk/images/trunk
    #valid=../trunk/images/trunk
    #names=/home/PycharmProjects/tensorrt/yolov3_arch/data/wings.names
        
    names = '\n'.join(labels)
    name_names = f'{proj_name}' + '.names'
    path_to_names = osp.join(data_dir, name_names)
    with open(path_to_names, 'w') as f:
        f.write(names)

    data = f'classes={len(labels)}\n'
    data += f'train={os.path.abspath(images_dir)}\n'
    data += f'valid={os.path.abspath(images_dir)}\n'
    data += f'names={os.path.abspath(path_to_names)}'

    data_name = f'{proj_name}' + '.data'
    path_to_data = osp.join(data_dir, data_name)
    with open(path_to_data, 'w') as f:
        print(path_to_data)
        f.write(data)


In [ ]:
import http.server
import socketserver
import json
import os
import os.path as osp

import cv2
import numpy as np
import pickle
import struct
import socket         


PORT = 6000

class HTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    """Extend SimpleHTTPRequestHandler to handle PUT requests"""
    root_dir = '/home/PycharmProjects/tensorrt'
    net_dir = os.path.join(root_dir, 'yolov3_arch')
    
    def do_PUT(self):
        """Save a file following a HTTP PUT request"""
        %cd {self.root_dir}
        print(self.headers)
        filename = self.headers['filename']
        self.data_dir, filename = filename.split('\\')
        
        if not os.path.isdir(self.data_dir):
            os.makedirs(self.data_dir)
            
        src_dir = osp.join(self.data_dir, 'src')
        if not os.path.isdir(src_dir):
            os.makedirs(src_dir)
        
        path_to_save = os.path.join(src_dir, filename)
        file_length = int(self.headers['Content-Length'])
        with open(path_to_save, 'wb') as output_file:
            output_file.write(self.rfile.read(file_length))
        self.send_response(201, 'Created')
        self.end_headers()
        reply_body = 'Saved "%s"\n' % path_to_save
        self.wfile.write(reply_body.encode('utf-8'))
        
    def do_TRAIN(self):
        %cd {self.root_dir}
        
        # convert data from labelme to yolo
        print(f'converting data to yolo with headers\n {self.headers}')
        
        data_dir = self.headers['data_dir']
        labels = self.headers['labels']
        labels = [l.strip() for l in labels.split(',') if l!='']
        
        assert len(os.listdir(data_dir))>=1, f'no files in {data_dir}' 

        labelme_to_yolo(data_dir, labels)
        
        # start training
        
        src_dir = osp.join(self.root_dir, data_dir, 'src')
        batch_size = len(os.listdir(src_dir))
        
        %cd {self.net_dir}
        print(f'start training')
        # comment line 415 in train.py
        #    check_git_status()
        print(f'start training with batch size {batch_size}')
        
        !python3 train.py \
        --cfg cfg/yolov3-1cls.cfg \
        --data /home/PycharmProjects/tensorrt/{data_dir}/{data_dir}.data \
        --weights weights/yolov3.pt \
        --img 480 \
        --batch {batch_size} \
        --epochs 300
#        --notest

        print(f'converting weights to yolo format')
        from models import convert
        convert('cfg/yolov3-1cls.cfg', 
                'weights/best.pt')
        
        # sending weights to jetson
        HOST = '172.25.207.82'  # The server's hostname or IP address
        W_PORT = 6000        # The port used by the server

        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, W_PORT))
            with open('weights/best.weights', 'rb') as f:
                data = f.read()
                
            print(len(data))
            message = struct.pack("Q", len(data)) + data
            s.sendall(message)

        print('finished')
        
        
        


Handler = HTTPRequestHandler

with socketserver.TCPServer(('', PORT), Handler) as httpd:
    print("serving at port", PORT)
    httpd.serve_forever()

serving at port 6000
